# Lexicographic Constraint Optimization (LCO) — Colab Simulation Notebook
### Companion Notebook for: *Procedures for Executing LCO Simulations Using Google Colab*
Author: **Antonios Valamontes** — Kapodistrian Academy of Science

This notebook implements the static two-tier LCO demo:
- Tier $\mathcal{L}_2$: maximize expected revenue
- Tier $\mathcal{L}_3$: minimize expected overbooking slack under a fixed revenue floor.

It is designed to run directly in Google Colab and reproduce the results described in the LCO procedures paper.

In [ ]:
# Install Pyomo and solvers (Colab-friendly)
!pip install pyomo highspy
!apt-get install -y coinor-cbc

In [ ]:
from pyomo.environ import *
import numpy as np
import pandas as pd


## Synthetic Booking Dataset
We use the same 10-rooms × 5-days synthetic instance as in the LCO hotel paper:
12 bookings with heterogeneous stay lengths, prices, and show probabilities.

In [ ]:
# Parameters: (start_day, length_of_stay, price_per_night, show_probability)
bookings = {
    1: (1,2,120,0.92),
    2: (1,3,110,0.85),
    3: (2,2,150,0.90),
    4: (2,3,130,0.80),
    5: (3,2,140,0.88),
    6: (3,3,100,0.83),
    7: (4,2,160,0.87),
    8: (4,2,115,0.78),
    9: (5,1,200,0.95),
    10: (1,1,180,0.90),
    11: (2,1,170,0.82),
    12: (3,1,175,0.89),
}


## Build Pyomo L2→L3 Model
Implements:
- Room exclusivity
- Acceptance/assignment linking
- Same-room continuity
- Overbooking slack per day (expected shows − capacity)

Tier $\mathcal{L}_2$ objective: maximize expected revenue.
Tier $\mathcal{L}_3$ objective: minimize total overbooking slack with the L2 revenue floor.

In [ ]:
def build_model():
    m = ConcreteModel()

    m.D = RangeSet(1,5)
    m.R = RangeSet(1,10)
    m.B = RangeSet(1,12)

    m.start = Param(m.B, initialize={b: bookings[b][0] for b in m.B})
    m.len   = Param(m.B, initialize={b: bookings[b][1] for b in m.B})
    m.price = Param(m.B, initialize={b: bookings[b][2] for b in m.B})
    m.showp = Param(m.B, initialize={b: bookings[b][3] for b in m.B})

    CAP = {d: 10 for d in m.D}
    m.cap = Param(m.D, initialize=CAP)

    def stay_days(b):
        s = int(m.start[b]); L = int(m.len[b])
        return [d for d in m.D if s <= d < s + L]

    instay = {(b,d) for b in m.B for d in m.D if d in stay_days(b)}
    m.InStay = Set(dimen=2, initialize=instay)
    yidx = {(b,r,d) for (b,d) in instay for r in m.R}
    m.YIDX = Set(dimen=3, initialize=yidx)

    contpairs = {(b,r,d)
                 for b in m.B for r in m.R for d in m.D
                 if (b,d) in instay and (b,d+1) in instay}
    m.ContPair = Set(dimen=3, initialize=contpairs)

    m.a = Var(m.B, within=Binary)
    m.y = Var(m.YIDX, within=Binary)
    m.w = Var(m.D, within=NonNegativeReals)

    def room_excl(m,r,d):
        return sum(m.y[b,r,d] for b in m.B if (b,d) in m.InStay) <= 1
    m.RoomExcl = Constraint(m.R,m.D,rule=room_excl)

    def assign_link(m,b,d):
        if (b,d) not in m.InStay:
            return Constraint.Skip
        return sum(m.y[b,r,d] for r in m.R) == m.a[b]
    m.Assign = Constraint(m.B,m.D,rule=assign_link)

    def continuity(m,b,r,d):
        return m.y[b,r,d] == m.y[b,r,d+1]
    m.Continuity = Constraint(m.ContPair, rule=continuity)

    def overbooking(m,d):
        exp_shows = sum(m.a[b]*m.showp[b] for b in m.B if (b,d) in m.InStay)
        return m.w[d] >= exp_shows - m.cap[d]
    m.Overbook = Constraint(m.D, rule=overbooking)

    m.Rev = sum(m.a[b]*m.price[b]*m.len[b] for b in m.B)
    m.obj = Objective(expr=m.Rev, sense=maximize)

    return m


## Solve Tier $\mathcal{L}_2$ (Revenue Maximization)
We first maximize expected revenue and record the optimum $Z_2^\ast$.

In [ ]:
m = build_model()
solver = SolverFactory('highs')
results_L2 = solver.solve(m, tee=False)
Z2 = float(m.Rev())
print('Tier L2 (Revenue) optimal value Z2* =', Z2)


## Solve Tier $\mathcal{L}_3$ (Overbooking Slack Minimization) Under the L2 Floor
We fix a revenue floor $\mathcal{L}_2(\mathbf{x}) \ge Z_2^\ast - \varepsilon_2$ and re-solve
to minimize the total overbooking slack $\sum_d w_d$.

In [ ]:
m.RevenueFloor = Constraint(expr=m.Rev >= Z2 - 1e-6)

m.del_component(m.obj)
m.obj = Objective(expr=sum(m.w[d] for d in m.D), sense=minimize)

results_L3 = solver.solve(m, tee=False)
Z3 = sum(float(m.w[d]()) for d in m.D)
print('Tier L3 (Overbooking slack) optimum sum =', Z3)


## Extract Accepted Bookings and Room Assignments
This summarizes which bookings are accepted and to which room they are assigned
after the two-tier lexicographic solve.

In [ ]:
assignments = []
for b in m.B:
    if m.a[b]() > 0.5:
        sdays = [d for d in m.D if (b,d) in m.InStay]
        room = None
        for r in m.R:
            if all((b,r,d) in m.YIDX and m.y[b,r,d]() > 0.5 for d in sdays):
                room = r
                break
        assignments.append((int(b), room, list(map(int,sdays))))

assignments_df = pd.DataFrame(assignments, columns=['Booking','Room','StayDays'])
assignments_df

## KPI Summary
We report the Tier $\mathcal{L}_2$ revenue optimum and the Tier $\mathcal{L}_3$ slack optimum.

In [ ]:
summary_df = pd.DataFrame({
    'L2_Revenue_Optimum_Z2*': [Z2],
    'L3_Overbooking_Slack_Sum_Z3': [Z3],
})
summary_df

---
### Reproducibility Note
This notebook is a companion artifact to the paper:

> Antonios Valamontes, *Procedures for Executing Lexicographic Constraint Optimization (LCO) Simulations Using Google Colab*, Kapodistrian Academy of Science.

All results here can be regenerated by running the cells in order in Google Colab.